In [1]:
import numpy as np
import pandas as pd
import os

In [17]:
# set parent folder as working directory
os.chdir("..")
os.getcwd()

'c:\\Users\\isaldiviagonzatti\\OneDrive - Kiel Institut für Weltwirtschaft\\Documents\\General\\Ignacio'

In [20]:
# define paths, create output folder if not there
data_path = 'data'
output_path = 'output'

if not os.path.exists(output_path):
	os.makedirs(output_path)

#### Country names dict

In [21]:
country_to_abbrev = {
    "Andorra": "AD",
    "United Arab Emirates": "AE",
    "Afghanistan": "AF",
    "Antigua and Barbuda": "AG",
    "Anguilla": "AI",
    "Albania": "AL",
    "Armenia": "AM",
    "Angola": "AO",
    "Antarctica": "AQ",
    "Argentina": "AR",
    "American Samoa": "AS",
    "Austria": "AT",
    "Australia": "AU",
    "Aruba": "AW",
    "Åland Islands": "AX",
    "Azerbaijan": "AZ",
    "Bosnia and Herzegovina": "BA",
    "Barbados": "BB",
    "Bangladesh": "BD",
    "Belgium": "BE",
    "Burkina Faso": "BF",
    "Bulgaria": "BG",
    "Bahrain": "BH",
    "Burundi": "BI",
    "Benin": "BJ",
    "Saint Barthélemy": "BL",
    "Bermuda": "BM",
    "Brunei Darussalam": "BN",
    "Bolivia (Plurinational State of)": "BO",
    "Bonaire, Sint Eustatius and Saba": "BQ",
    "Brazil": "BR",
    "Bahamas": "BS",
    "Bhutan": "BT",
    "Bouvet Island": "BV",
    "Botswana": "BW",
    "Belarus": "BY",
    "Belize": "BZ",
    "Canada": "CA",
    "Cocos (Keeling) Islands": "CC",
    "Congo, Democratic Republic of the": "CD",
    "Central African Republic": "CF",
    "Congo": "CG",
    "Switzerland": "CH",
    "Côte d'Ivoire": "CI",
    "Cook Islands": "CK",
    "Chile": "CL",
    "Cameroon": "CM",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Cuba": "CU",
    "Cabo Verde": "CV",
    "Curaçao": "CW",
    "Christmas Island": "CX",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Germany": "DE",
    "Djibouti": "DJ",
    "Denmark": "DK",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Algeria": "DZ",
    "Ecuador": "EC",
    "Estonia": "EE",
    "Egypt": "EG",
    "Western Sahara": "EH",
    "Eritrea": "ER",
    "Spain": "ES",
    "Ethiopia": "ET",
    "Finland": "FI",
    "Fiji": "FJ",
    "Falkland Islands (Malvinas)": "FK",
    "Micronesia (Federated States of)": "FM",
    "Faroe Islands": "FO",
    "France": "FR",
    "Gabon": "GA",
    "United Kingdom of Great Britain and Northern Ireland": "UK", #original is GB, Eurostat uses UK
    "United Kingdom of GB": "GB",
    "Grenada": "GD",
    "Georgia": "GE",
    "French Guiana": "GF",
    "Guernsey": "GG",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greenland": "GL",
    "Gambia": "GM",
    "Guinea": "GN",
    "Guadeloupe": "GP",
    "Equatorial Guinea": "GQ",
    "Greece": "EL", #original ir GR, Eurostat uses EL
    "South Georgia and the South Sandwich Islands": "GS",
    "Guatemala": "GT",
    "Guam": "GU",
    "Guinea-Bissau": "GW",
    "Guyana": "GY",
    "Hong Kong": "HK",
    "Heard Island and McDonald Islands": "HM",
    "Honduras": "HN",
    "Croatia": "HR",
    "Haiti": "HT",
    "Hungary": "HU",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Isle of Man": "IM",
    "India": "IN",
    "British Indian Ocean Territory": "IO",
    "Iraq": "IQ",
    "Iran (Islamic Republic of)": "IR",
    "Iceland": "IS",
    "Italy": "IT",
    "Jersey": "JE",
    "Jamaica": "JM",
    "Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "Cambodia": "KH",
    "Kiribati": "KI",
    "Comoros": "KM",
    "Saint Kitts and Nevis": "KN",
    "Korea (Democratic People's Republic of)": "KP",
    "Korea, Republic of": "KR",
    "Kuwait": "KW",
    "Cayman Islands": "KY",
    "Kazakhstan": "KZ",
    "Lao People's Democratic Republic": "LA",
    "Lebanon": "LB",
    "Saint Lucia": "LC",
    "Liechtenstein": "LI",
    "Sri Lanka": "LK",
    "Liberia": "LR",
    "Lesotho": "LS",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Latvia": "LV",
    "Libya": "LY",
    "Morocco": "MA",
    "Monaco": "MC",
    "Moldova, Republic of": "MD",
    "Montenegro": "ME",
    "Saint Martin (French part)": "MF",
    "Madagascar": "MG",
    "Marshall Islands": "MH",
    "North Macedonia": "MK",
    "Mali": "ML",
    "Myanmar": "MM",
    "Mongolia": "MN",
    "Macao": "MO",
    "Northern Mariana Islands": "MP",
    "Martinique": "MQ",
    "Mauritania": "MR",
    "Montserrat": "MS",
    "Malta": "MT",
    "Mauritius": "MU",
    "Maldives": "MV",
    "Malawi": "MW",
    "Mexico": "MX",
    "Malaysia": "MY",
    "Mozambique": "MZ",
    "Namibia": "NA",
    "New Caledonia": "NC",
    "Niger": "NE",
    "Norfolk Island": "NF",
    "Nigeria": "NG",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "Nepal": "NP",
    "Nauru": "NR",
    "Niue": "NU",
    "New Zealand": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "French Polynesia": "PF",
    "Papua New Guinea": "PG",
    "Philippines": "PH",
    "Pakistan": "PK",
    "Poland": "PL",
    "Saint Pierre and Miquelon": "PM",
    "Pitcairn": "PN",
    "Puerto Rico": "PR",
    "Palestine, State of": "PS",
    "Portugal": "PT",
    "Palau": "PW",
    "Paraguay": "PY",
    "Qatar": "QA",
    "Réunion": "RE",
    "Romania": "RO",
    "Serbia": "RS",
    "Russian Federation": "RU",
    "Rwanda": "RW",
    "Saudi Arabia": "SA",
    "Solomon Islands": "SB",
    "Seychelles": "SC",
    "Sudan": "SD",
    "Sweden": "SE",
    "Singapore": "SG",
    "Saint Helena, Ascension and Tristan da Cunha": "SH",
    "Slovenia": "SI",
    "Svalbard and Jan Mayen": "SJ",
    "Slovakia": "SK",
    "Sierra Leone": "SL",
    "San Marino": "SM",
    "Senegal": "SN",
    "Somalia": "SO",
    "Suriname": "SR",
    "South Sudan": "SS",
    "Sao Tome and Principe": "ST",
    "El Salvador": "SV",
    "Sint Maarten (Dutch part)": "SX",
    "Syrian Arab Republic": "SY",
    "Eswatini": "SZ",
    "Turks and Caicos Islands": "TC",
    "Chad": "TD",
    "French Southern Territories": "TF",
    "Togo": "TG",
    "Thailand": "TH",
    "Tajikistan": "TJ",
    "Tokelau": "TK",
    "Timor-Leste": "TL",
    "Turkmenistan": "TM",
    "Tunisia": "TN",
    "Tonga": "TO",
    "Turkey": "TR",
    "Trinidad and Tobago": "TT",
    "Tuvalu": "TV",
    "Taiwan, Province of China": "TW",
    "Tanzania, United Republic of": "TZ",
    "Ukraine": "UA",
    "Uganda": "UG",
    "United States Minor Outlying Islands": "UM",
    "United States of America": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Holy See": "VA",
    "Saint Vincent and the Grenadines": "VC",
    "Venezuela (Bolivarian Republic of)": "VE",
    "Virgin Islands (British)": "VG",
    "Virgin Islands (U.S.)": "VI",
    "Viet Nam": "VN",
    "Vanuatu": "VU",
    "Wallis and Futuna": "WF",
    "Samoa": "WS",
    "Yemen": "YE",
    "Mayotte": "YT",
    "South Africa": "ZA",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}
    
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))

In [22]:
countries=['Belgium','Bulgaria','Cyprus', 'Greece','Germany','Croatia','Italy', 
           'Denmark','Estonia','Spain','Finland','France','Ireland','Lithuania',
           'Latvia','Malta','Netherlands','Poland','Portugal', 'Romania',
           'Sweden','United Kingdom of Great Britain and Northern Ireland']


In [23]:
countryAbb = [x if x not in country_to_abbrev else country_to_abbrev[x] for x in countries]

### SDG Official Data

In [24]:
# SDG14 indicators from the UNstats
# https://unstats.un.org/sdgs/dataportal/database
 
sdg14 = pd.read_excel(os.path.join(data_path, "Goal14_raw.xlsx"), sheet_name=1)

In [25]:
# filter columns 
sdg14 = sdg14[['Target', 'Indicator','SeriesCode', 'GeoAreaName',
                   'TimePeriod', 'Value','Units','SeriesDescription', ]]

In [26]:
# get name used for UK
sdg14.GeoAreaName[sdg14['GeoAreaName'].str.contains("United")].unique()

array(['United Kingdom of Great Britain and Northern Ireland'],
      dtype=object)

In [27]:
# filter countries
sdg14 = sdg14[sdg14['GeoAreaName'].isin(countries)]
# show indicators
pd.DataFrame(sdg14.groupby(['Indicator', 'SeriesCode', 'SeriesDescription', 'Units']).size())

0
Indicator SeriesCode        SeriesDescription                                  Units       
14.1.1    EN_MAR_BEALITSQ   Beach litter per square kilometer (Number)         NUMBER    90
          EN_MAR_BEALIT_BP  Beach litter originating from national land-bas... PERCENT  110
          EN_MAR_BEALIT_BV  Beach litter originating from national land-bas... TONNES   110
          EN_MAR_BEALIT_EXP Exported beach litter originating from national... TONNES   110
          EN_MAR_BEALIT_OP  Beach litter originating from national land-bas... PERCENT  110
          EN_MAR_BEALIT_OV  Beach litter originating from national land-bas... TONNES   110
          EN_MAR_CHLANM     Chlorophyll-a anomaly, remote sensing (%)          PERCENT  147
          EN_MAR_CHLDEV     Chlorophyll-a deviations, remote sensing (%)       PERCENT  372
14.2.1    EN_SCP_ECSYBA     Number of countries using ecosystem-based appro... NUMBER    10
14.3.1    ER_OAW_MNACD      Average marine acidity (pH) measured at agreed ... PH       434
14.4.1    ER_H2O_FWTL       Proportion of fish stocks within biologically s... PERCENT   23
14.5.1    ER_MRN_MARIN      Coverage of protected areas in relation to mari... PERCENT   22
          ER_MRN_MARINT     Protected marine area (Exclusive Economic Zones... KMSQ      22
          ER_MRN_MPA        Average proportion of Marine Key Biodiversity A... PERCENT  484
14.6.1    ER_REG_UNFCIM     Progress by countries in the degree of implemen... NUMBER    66
14.7.1    EN_SCP_FSHGDP     Sustainable fisheries as a proportion of GDP       PERCENT  440
14.a.1    ER_RDE_OSEX       National ocean science expenditure as a share o... PERCENT   48
14.b.1    ER_REG_SSFRAR     Degree of application of a legal/regulatory/pol... NUMBER    66
14.c.1    ER_UNCLOS_IMPLE   Score for the implementation of UNCLOS and its ... PERCENT   11
          ER_UNCLOS_RATACC  Score for the ratification of and accession to ... PERCENT   11

In [44]:
sdg14EZZ = sdg14[sdg14['SeriesCode']=='ER_REG_SSFRAR'].pivot_table(columns='TimePeriod', index='GeoAreaName', values='Value', aggfunc='mean')

sdg14EZZ
#[[2012,2014,2021]]

TimePeriod,2018,2020,2022
GeoAreaName,,,
Belgium,4.0,4.0,5.0
Bulgaria,4.0,4.0,5.0
Croatia,4.0,4.0,5.0
Cyprus,4.0,4.0,5.0
Denmark,4.0,4.0,5.0
Estonia,4.0,4.0,5.0
Finland,4.0,NaN,5.0
France,4.0,4.0,5.0
Germany,4.0,4.0,5.0


### 14.1

#### (b) Floating Plastic Debris Density


In [30]:
# Plastic Waste kg/ha, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_WASGEN/

wasteG = pd.read_csv(os.path.join(data_path,'env_wasgen.csv')) #read file
wasteG = wasteG[wasteG['geo'].isin(countryAbb)] #filter countries
wasteG['geo'] = wasteG['geo'].map(abbrev_to_country).fillna(wasteG['geo']) #change abb to name
wasteG  = wasteG[wasteG['unit']=='KG_HAB'].pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean') #pivot table 

mr = wasteG.columns[-1] # most recent year
wasteG
#maxMin transformation as (x-max)/(max-min) * 100 --> converts to 0-100 scale with 0=worst, 100=best
(wasteG[[2012, 2014, mr]].max().max() - wasteG[[2012,2014, mr]])/(wasteG[[2012,2014, mr]].max().max() - wasteG[[2012,2014, mr]].min().min())*100 

TIME_PERIOD,2012,2014,2020
geo,,,
Belgium,54.621849,0.000000,31.932773
Bulgaria,92.436975,81.512605,59.663866
Croatia,96.638655,94.117647,78.151261
Cyprus,100.000000,99.159664,88.235294
Denmark,88.235294,89.075630,84.033613
Estonia,89.915966,83.193277,77.310924
Finland,89.915966,86.554622,84.033613
France,83.193277,82.352941,73.949580
Germany,78.151261,76.470588,73.109244


In [32]:
# Recovery rates for packaging waste, Plastic Packaging, Percentage, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ten00062/

wasteR = pd.read_csv(os.path.join(data_path,'ten00062.csv'))
wasteR = wasteR[wasteR['geo'].isin(countryAbb)]
wasteR['geo'] = wasteR['geo'].map(abbrev_to_country).fillna(wasteR['geo'])
wasteR = wasteR.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

mr = wasteR.columns[-1] # most recent year
wasteR = wasteR.ffill(axis = 1) #fill empty values with last available year
wasteR[[2012, 2014, mr]]

TIME_PERIOD,2012,2014,2020
geo,,,
Belgium,92.7,98.3,99.4
Bulgaria,44.1,64.1,50.6
Croatia,45.4,38.0,34.1
Cyprus,44.8,46.6,51.9
Denmark,99.4,98.2,73.1
Estonia,44.0,75.9,87.4
Finland,51.0,68.3,99.4
France,64.0,64.0,71.8
Germany,99.7,99.8,99.9


#### (a) Index of coastal eutrophication


In [33]:
#Gross nutrient balance (Nitrogen kg/ha), Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/AEI_PR_GNB__custom_153613/

nitro = pd.read_csv(os.path.join(data_path,'aei_pr_gnb.csv'))
nitro = nitro[nitro['geo'].isin(countryAbb)]
nitro['geo'] = nitro['geo'].map(abbrev_to_country).fillna(nitro['geo'])
nitro.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

TIME_PERIOD,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
geo,,,,,,,,,,,,,,,,,,,,,
Belgium,254.0,241.0,232.0,233.0,225.0,189.9,180.0,169.5,151.6,145.6,...,129.3,142.0,143.2,143.1,137.9,131.7,132.0,NaN,NaN,NaN
Bulgaria,29.0,42.0,33.0,21.0,30.0,28.2,26.8,21.6,31.3,18.9,...,20.3,14.2,11.9,24.2,15.8,27.6,28.0,71.10,65.80,61.8
Croatia,NaN,NaN,NaN,NaN,NaN,122.8,122.8,109.3,119.4,111.1,...,61.6,80.8,93.2,86.3,43.9,48.3,59.7,35.40,65.10,57.6
Cyprus,207.0,211.0,216.0,214.0,197.0,186.0,191.2,209.0,181.1,172.3,...,177.6,191.4,198.7,184.4,178.5,194.3,194.0,NaN,NaN,NaN
Denmark,156.0,139.0,132.0,132.0,125.0,131.7,127.4,124.4,117.5,121.3,...,86.5,90.0,88.0,83.4,87.1,79.8,80.0,NaN,NaN,NaN
Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.5,...,25.1,30.8,32.4,28.0,NaN,NaN,NaN,NaN,NaN,NaN
Finland,79.0,71.0,66.0,74.0,64.0,54.9,59.1,54.8,55.8,53.1,...,38.0,57.1,50.2,47.5,46.5,47.5,49.5,47.40,51.50,58.0
France,55.0,58.0,53.0,51.0,55.0,42.6,49.5,39.6,61.0,35.6,...,29.8,36.3,49.1,23.5,29.7,29.6,37.3,47.50,33.70,40.8
Germany,109.0,106.0,99.0,97.0,102.0,110.0,97.5,100.7,113.7,84.5,...,65.4,77.7,88.6,75.1,78.9,65.9,81.8,69.05,61.51,77.9


### 14.3

####  Average marine acidity (pH) measured at agreed suite of representative sampling stations

In [34]:
# Population on 1 January, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/tps00001/

pop = pd.read_csv(os.path.join(data_path,'tps00001.csv'))
pop = pop[pop['geo'].isin(countryAbb)]
pop['geo'] = pop['geo'].map(abbrev_to_country).fillna(pop['geo'])
pop = pop.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')


In [35]:
# Greenhouse gas emissions under the Effort Sharing Decision (ESD), Million tonnes CO2 equivalent (Mt CO2 eq), European Environment Agency
# https://www.eea.europa.eu/data-and-maps/data/esd-4

ghgESD = pd.read_excel(os.path.join(data_path,'EEA_ESD-GHG_2022.xlsx'), sheet_name=1, skiprows=11)
ghgESD = ghgESD[ghgESD['Geographic entity'].isin(countries)]
ghgESD = ghgESD.set_index('Geographic entity')
ghgESD = ghgESD.dropna(axis=1)

mr = ghgESD.columns[-1] # most recent year
ghgESD[[2012, 2016, mr]]

,2012,2016,2021
Geographic entity,,,
Belgium,73.941551,74.063149,69.526211
Bulgaria,23.239884,25.587947,25.480952
Croatia,16.280625,16.006813,16.233205
Cyprus,4.250998,4.111441,4.604652
Denmark,35.677561,33.124678,29.372533
Estonia,6.475652,6.218046,6.134968
Finland,31.976059,31.358144,27.193424
France,365.369122,351.924668,327.477527
Germany,438.770433,454.157411,405.654586


In [36]:
# CO2, KG_HAB, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_AC_AINAH_R2

co2 = pd.read_csv(os.path.join(data_path,'env_ac_ainah_r2.csv'))
co2 = co2[co2['geo'].isin(countryAbb)]
co2['geo'] = co2['geo'].map(abbrev_to_country).fillna(co2['geo'])
co2 = co2.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

mr = co2.columns[-1] # most recent year
co2[[2012, 2016, mr]]/1000

TIME_PERIOD,2012,2016,2021
geo,,,
Belgium,6.598775,6.285137,5.892694
Bulgaria,6.452205,6.249124,5.921821
Croatia,3.313530,3.130383,2.967264
Cyprus,6.612553,6.587503,6.021738
Denmark,12.735182,11.728105,8.915469
Estonia,13.444888,12.789470,7.573749
Finland,8.914348,8.179107,6.142578
France,3.872669,3.531338,3.130740
Germany,8.635811,8.097228,6.412099


In [37]:
# CO2, TOTX4_MEMONIA, THS_T thousand tonnes, Eurostat
# https://ec.europa.eu/eurostat/databrowser/view/ENV_AIR_GGE/

co2 = pd.read_csv(os.path.join(data_path,'env_air_gge.csv'))
co2 = co2[co2['geo'].isin(countryAbb)]
co2['geo'] = co2['geo'].map(abbrev_to_country).fillna(co2['geo'])
co2 = co2.pivot_table(columns='TIME_PERIOD', index='geo', values='OBS_VALUE', aggfunc='mean')

mr = co2.columns[-1] # most recent year
#co2[[2012, 2016, mr]] /1000  # million metric tonnes

co2pc = co2[[2012, 2016, 2020]]/pop[[2012, 2016, 2020]]*1000  ## tonnes co2 per capita 
co2pc

TIME_PERIOD,2012,2016,2020
geo,,,
Belgium,9.610486,9.192649,8.155780
Bulgaria,6.663881,6.447040,5.377837
Croatia,4.541044,4.410935,4.197550
Cyprus,9.398094,9.735017,8.561990
Denmark,7.658618,7.034621,5.065501
Estonia,13.563281,13.448315,7.084492
Finland,9.830334,8.967016,6.973570
France,5.875707,5.432116,4.407393
Germany,10.441041,10.061325,7.851746


In [38]:
# Fossil CO2 emissions by country (territorial), million tonnes of carbon per year (1MtC = 1 million tonne of carbon = 3.664 million tonnes of CO2)
# https://globalcarbonbudget.org/carbonbudget/

co2T = pd.read_excel(os.path.join(data_path,'National_Fossil_Carbon_Emissions_2022v1.0.xlsx'), sheet_name=1, skiprows=11) 
co2T = co2T.T
co2T.columns = co2T.iloc[0,:]
co2T.columns = co2T.columns.astype(int)
co2T = co2T.rename_axis(index='geo', columns=None)
co2T = co2T[co2T.index.isin(countries)]

mr = co2T.columns[-1] # most recent year
co2T = co2T[[2012, 2016, mr]] * 3.664 # convert from carbon to co2 

co2pc = co2T/pop[[2012, 2016, 2021]]*1000_000  ## tonnes co2 per capita 
co2pc

,2012,2016,2021
geo,,,
Belgium,9.241302,8.805640,8.284225
Bulgaria,6.597165,6.358077,6.153389
Croatia,4.463862,4.321271,4.385504
Cyprus,8.426844,8.694153,8.482547
Denmark,7.144668,6.488809,5.064508
Estonia,13.427859,13.343541,7.856005
Finland,9.469202,8.598362,6.795037
France,5.467664,5.018725,4.522298
Germany,10.129644,9.739375,8.114405


### 14.4

####   Proportion of fish stocks within biologically sustainable levels

### 14.5

####  Coverage of protected areas in relation to marine area

In [39]:
# Marine protected areas (% of territorial waters), World Bank aggregation of https://www.protectedplanet.net/en/thematic-areas/marine-protected-areas
# https://data.worldbank.org/indicator/ER.MRN.PTMR.ZS

mpa = pd.read_csv(os.path.join(data_path,'API_ER.MRN.PTMR.ZS_DS2.csv'), skiprows=4)
mpa = mpa[mpa['Country Name'].isin(countries)]
mpa = mpa.set_index('Country Name')
mpa = mpa.dropna(axis=1)
mpa = mpa[['2016', '2021']]

mpa = (mpa/0.3).round(2)  # dis-ref with target 30%
mpa[mpa>100] = 100
mpa

,2016,2021
Country Name,,
Belgium,100.00,100.00
Bulgaria,27.01,27.03
Cyprus,0.41,28.74
Germany,100.00,100.00
Denmark,59.49,61.07
Spain,29.10,42.52
Estonia,62.09,62.60
Finland,37.10,39.96
France,87.22,100.00


In [40]:
# OHI Index 
# https://oceanhealthindex.org/global-scores/data-download/

ohi = pd.read_csv(os.path.join(data_path,'scoresOHI.csv'))
ohi = ohi[ohi['region_name'].isin(countries)]
ohi = ohi[(ohi.long_goal == 'Biodiversity')  & (ohi.dimension == 'score') ]
ohi = ohi.pivot_table( columns='scenario', index='region_name', values='value', aggfunc='mean')

ohi[[2012, 2016, 2022]]


scenario,2012,2016,2022
region_name,,,
Belgium,74.65,70.37,71.53
Bulgaria,76.12,67.76,66.19
Croatia,60.77,54.34,55.65
Cyprus,68.48,66.02,64.07
Denmark,72.56,70.68,71.17
Estonia,80.68,75.47,74.96
Finland,79.25,72.17,73.02
France,72.20,69.96,75.08
Germany,73.45,69.15,72.25


### 14.6

####  Degree of implementation of international instruments aiming to combat illegal, unreported and unregulated fishing

In [41]:
# Fisheries Support Estimate
# https://stats.oecd.org/Index.aspx?DataSetCode=FISH_FSE

fse = pd.read_csv(os.path.join(data_path,'FISH_FSE.csv'))
fse = fse[fse['Country'].isin(countries)]
# fse = fse[(fse.long_goal == 'Biodiversity')  & (fse.dimension == 'score') ]
# fse = fse.pivot_table( columns='scenario', index='region_name', values='value', aggfunc='mean')

'I.E.1. Fuel tax concessions'

# e i) transfers to individual fishers (including market prize support and fuel tax concessions) and ii) general 
# service support transfers (excluding transfers for the management of resources, education and training, 
# and research and development.) 


# fse[[2012, 2016, 2022]]

fse.Variable.unique()

array(['I.A. Transfers based on input use',
       'I.A.2. Transfers based on fixed capital formation',
       'I.A.2.1.Support to vessel construction/purchase',
       'I.A.2.2.Support to modernisation',
       'II.B. Provision of infrastructure',
       'II.B.1. Capital expenditures', 'II.C. Marketing and promotion',
       'II.D. Support to fishing communities',
       'II.F. Research and development', 'II.G. Management of resources',
       'II.G.2. Stock enhancement programs',
       'II.H. Miscellaneous support for services to the sector',
       'I.E. Tax exemptions', 'I.E.1. Fuel tax concessions',
       'I.C. Transfers based on the reduction of productive capacity',
       'I.D. Miscellaneous direct support to individuals and companies',
       'II.G.1. Management expenditures',
       'II.G.3. Enforcement expenditures',
       'III.C. Payments made by the fisheries sector, for management, research and enforcement',
       'I.B. Transfers based on fishers income', 'I.B.1. Inco

In [42]:
# TAC/Catch

### 14.7

####  Sustainable fisheries as a proportion of GDP in small island developing States, least developed countries and all countries

### 14.a

#### Proportion of total research budget allocated to research in the field of marine technology

### 14.b

#### Degree of application of a legal/regulatory/policy/institutional framework which recognizes and protects access rights for small‐scale fisheries